In [1]:
from google.colab import drive
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from scipy.stats import mode

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load models
model_paths = [
    ("/content/drive/MyDrive/FYP/combine_model/mobilenet/1.keras", (256, 256), "mobilenet"),
    ("/content/drive/MyDrive/FYP/combine_model/nasnetMobile/1.keras", (256, 256), "nasnetmobile"),
    ("/content/drive/MyDrive/FYP/combine_model/vgg/2.keras", (256, 256), "vgg")
]

models = [load_model(path) for path, _, _ in model_paths]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
# Create an ImageDataGenerator for test data
test_data_dir = "/content/drive/MyDrive/FYP/split/test"
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Get true labels
y_true = test_generator.classes

Found 131 images belonging to 2 classes.


In [7]:
# Generate predictions from base models
base_predictions = []
for model in models:
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    base_predictions.append(y_pred_classes)

# Stack predictions and apply max voting
base_predictions = np.array(base_predictions)
#The following line was changed.
final_predictions = mode(base_predictions, axis=0, keepdims=True).mode.flatten() # Flatten the result to a 1D array

5/5 ━━━━━━━━━━━━━━━━━━━━ 93s 18s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 125s 23s/step


In [8]:
# Calculate accuracy
max_voting_accuracy = accuracy_score(y_true, final_predictions)
print(f"Accuracy for Max Voting Ensemble: {max_voting_accuracy}")

Accuracy for Max Voting Ensemble: 0.9083969465648855
